In [0]:
%tensorflow_version  2.x
! pip install -q keras==2.3.0

     |████████████████████████████████| 378kB 4.8MB/s 


In [0]:
# Per sapere che GPU il signor Google ci regala sta volta

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9583200720809256018, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 14603007595597933249
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 2510438938007864513
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15701401920
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 483586610726419763
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import make_scorer, classification_report
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.multiclass import OneVsRestClassifier


from xgboost.sklearn import XGBClassifier
import xgboost

from keras import models, layers, optimizers, metrics, callbacks


from keras.wrappers.scikit_learn import KerasClassifier
import collections

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Funzioni per calcolare ndcg

def dcg_score(y_true, y_score, k=5):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])

    gain = 2 ** y_true - 1

    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gain / discounts)


def ndcg_score(ground_truth, predictions, k=5):
    lb = LabelBinarizer()
    lb.fit(range(len(predictions) ))
    T = lb.transform(ground_truth)

    scores = []

    # Iterate over each y_true and compute the DCG score
    for y_true, y_score in zip(T, predictions):
        actual = dcg_score(y_true, y_score, k)
        best = dcg_score(y_true, y_true, k)
        score = float(actual) / float(best)
        scores.append(score)

    return np.mean(scores)



ndcg_scorer = make_scorer(ndcg_score, needs_proba=True, k=5)

In [0]:
# Funzione per standardizzare
def standardize(data, columns):
    for i in columns:
        data[i] = (data[i] - data[i].mean()) / data[i].std()
    return data

In [0]:
train = pd.read_csv('/content/drive/My Drive/progetto_aml/dataset/train_users_2.csv')
test = pd.read_csv("/content/drive/My Drive/progetto_aml/dataset/test_users.csv")
sessions = pd.read_csv("/content/drive/My Drive/progetto_aml/dataset/sessions.csv")

# Preprocessing

In [0]:
train_row = train.shape[0]
data = train.append(test)

In [0]:
# Scartiamo la feature "date_first_booking" inutile

data.drop("date_first_booking", axis = 1, inplace = True)

In [0]:
# Si prova a sistemare gli user che hanno inserito l'anno di nascita (il dataset risale al 2015)

user_with_year_age_mask = data['age'] > 1000
data.loc[user_with_year_age_mask, 'age'] = 2015 - data.loc[user_with_year_age_mask, 'age']

In [0]:
# Si pongono dei limiti all'età: sia assume che sotto i 18 anni e sopra i 100 siano errori


data.loc[(data['age'] > 100) | (data['age'] < 18), 'age'] = np.nan
data["age"] = (data["age"] - data["age"].mean()) / data["age"].std()

In [0]:
data["date_account_created"] = pd.to_datetime(data["date_account_created"])
data["timestamp_first_active"] = pd.to_datetime(data["timestamp_first_active"], format='%Y%m%d%H%M%S')

In [0]:
date_account_created = pd.DatetimeIndex(data['date_account_created'])
date_first_active = pd.DatetimeIndex(data['timestamp_first_active'])

In [0]:
# Split date in giorno, settimana, mese, anno

data['day_first_active'] = date_first_active.day
data['weekday_first_active'] = date_first_active.weekday
data['month_first_active'] = date_first_active.month
data['year_first_active'] = date_first_active.year

In [0]:
# Diffrenza tra data creazione account e prima attività eseguita
data["time_lag"] = (date_account_created - date_first_active).total_seconds().astype(int).values

In [0]:
#Features cicliche
data['month_first_active_sin'] = np.sin((data.month_first_active-1)*(2.*np.pi/12))
data['month_first_active_cos'] = np.cos((data.month_first_active-1)*(2.*np.pi/12))

data['day_first_active_sin'] = np.sin((data.day_first_active-1)*(2.*np.pi/31))
data['day_first_active_cos'] = np.cos((data.day_first_active-1)*(2.*np.pi/31))

data['weekday_first_active_sin'] = np.sin((data.weekday_first_active-1)*(2.*np.pi/7))
data['weekday_first_active_cos'] = np.cos((data.weekday_first_active-1)*(2.*np.pi/7))

In [0]:
standardize(data, ["year_first_active", "time_lag", "day_first_active", "weekday_first_active", "month_first_active"])

,id,date_account_created,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination,day_first_active,weekday_first_active,month_first_active,year_first_active,time_lag,month_first_active_sin,month_first_active_cos,day_first_active_sin,day_first_active_cos,weekday_first_active_sin,weekday_first_active_cos
0,gxn3p5htnn,2010-06-28,2009-03-19 04:32:55,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF,0.347279,0.115325,-1.158200,-4.603539,44.307172,0.866025,5.000000e-01,-0.485302,-0.874347,0.974928,-0.222521
1,820tgsjxq7,2011-05-25,2009-05-23 17:48:09,MALE,0.163287,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF,0.804052,1.142290,-0.488226,-4.603539,69.537221,0.866025,-5.000000e-01,-0.968077,-0.250653,-0.433884,-0.900969
2,4ft3gnwmtx,2010-09-28,2009-06-09 23:12:47,FEMALE,1.702840,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US,-0.794653,-0.911641,-0.153239,-4.603539,45.183726,0.500000,-8.660254e-01,0.998717,-0.050649,0.000000,1.000000
3,bjjt8pjhuk,2011-12-05,2009-10-31 06:01:29,FEMALE,0.505410,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other,1.717599,1.142290,1.186710,-4.603539,72.720420,-1.000000,-1.836970e-16,-0.201299,0.979530,-0.433884,-0.900969
4,87mebub9p4,2010-09-14,2009-12-08 06:11:05,-unknown-,0.419879,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US,-0.908847,-0.911641,1.856684,-4.603539,26.621924,-0.500000,8.660254e-01,0.988468,0.151428,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62091,cv0na2lf5a,2014-09-30,2014-09-30 23:52:32,-unknown-,-0.435428,basic,0,en,direct,direct,untracked,Web,Windows Desktop,IE,NaN,1.603405,-0.911641,0.851723,0.820888,-0.061335,-0.866025,-5.000000e-01,-0.394356,0.918958,0.000000,1.000000
62092,zp8xfonng8,2014-09-30,2014-09-30 23:53:06,-unknown-,NaN,basic,23,ko,direct,direct,untracked,Android,Android Phone,-unknown-,NaN,1.603405,-0.911641,0.851723,0.820888,-0.061372,-0.866025,-5.000000e-01,-0.394356,0.918958,0.000000,1.000000
62093,fa6260ziny,2014-09-30,2014-09-30 23:54:08,-unknown-,NaN,basic,0,de,direct,direct,linked,Web,Windows Desktop,Firefox,NaN,1.603405,-0.911641,0.851723,0.820888,-0.061441,-0.866025,-5.000000e-01,-0.394356,0.918958,0.000000,1.000000
62094,87k0fy4ugm,2014-09-30,2014-09-30 23:54:30,-unknown-,NaN,basic,0,en,sem-brand,google,omg,Web,Mac Desktop,Safari,NaN,1.603405,-0.911641,0.851723,0.820888,-0.061465,-0.866025,-5.000000e-01,-0.394356,0.918958,0.000000,1.000000


In [0]:
# Drop colonne duplicate

data.drop(["date_account_created", "timestamp_first_active",
           "day_first_active", "weekday_first_active",
           "month_first_active"], axis = 1, inplace = True)

In [0]:
# Alcuni valori pochi presenti nel dataset vengono cambiati in "other"

data["language"] = data["language"].replace(data["language"].value_counts()[data["language"].value_counts() < 10].index.to_list(), "other")

data["first_browser"] = data["first_browser"].replace(data["first_browser"].value_counts()[data["first_browser"].value_counts() < 60].index.to_list(), "other")

data["affiliate_provider"] = data["affiliate_provider"].replace(data["affiliate_provider"].value_counts()[data["affiliate_provider"].value_counts() < 100].index.to_list(), "other")

In [0]:
sessions.rename(columns = {'user_id': 'id'}, inplace=True)

In [0]:
sessions["action"] = sessions["action"].replace(sessions["action"].value_counts()[sessions["action"].value_counts() < 1000].index.to_list(), "other")

sessions["action_detail"] = sessions["action_detail"].replace(sessions["action_detail"].value_counts()[sessions["action_detail"].value_counts() < 1000].index.to_list(), "other")

In [0]:
action_count = sessions.groupby(['id', 'action'])['secs_elapsed'].count().unstack()
action_type_count = sessions.groupby(['id', 'action_type'])['secs_elapsed'].count().unstack()
action_detail_count = sessions.groupby(['id', 'action_detail'])['secs_elapsed'].count().unstack()
device_type_sum = sessions.groupby(['id', 'device_type'])['secs_elapsed'].sum().unstack()

sessions_data = pd.concat([action_count, action_type_count, action_detail_count, device_type_sum], axis=1)
sessions_data.columns = sessions_data.columns.map(lambda x: str(x) + '_count')


#Standardize and join
sessions_data = (sessions_data - sessions_data.mean()) / sessions_data.std()


# Device più usato
sessions_data['most_used_device'] = sessions.groupby('id')['device_type'].max()

data = data.join(sessions_data, on='id')

In [0]:
# Calcolo di diverse feature su "secs_elapsed"

a = sessions.groupby('id')['secs_elapsed'].agg(
    
        secs_elapsed_sum = np.sum,
        secs_elapsed_mean = np.mean,
        secs_elapsed_min = np.min,
        secs_elapsed_max = np.max,
        secs_elapsed_median = np.median,
        secs_elapsed_std = np.std,
        day_pauses = lambda x: (x > 86400).sum(),
        long_pauses = lambda x: (x > 300000).sum(),
        short_pauses = lambda x: (x < 3600).sum(),
        session_length = np.count_nonzero
)

a["secs_elapsed_skew"] = sessions.groupby("id")["secs_elapsed"].skew()
# Standardize
a = (a - a.mean()) / a.std()

In [0]:
data = data.join(a, on='id')

In [0]:
categorical_features = [
    'gender', 'signup_method', 'signup_flow', 'language',
    'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked',
    'signup_app', 'first_device_type', 'first_browser', 'most_used_device'
]
data = pd.get_dummies(data, columns=categorical_features)

In [0]:
data.fillna(-99, inplace = True)

In [0]:
train = data[:train_row]
test = data[train_row:]

# Modello XGBoost

In [0]:
X_train, X_test = train_test_split(train, test_size = 0.2, stratify = train["country_destination"], random_state = 42)

In [0]:
X_train.drop("id", axis=1, inplace=True)
X_test.drop("id", axis=1, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [0]:
train.drop("id", axis=1, inplace=True)
y_train = train.pop("country_destination")

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [0]:
y_train = X_train.pop("country_destination")
y_test = X_test.pop("country_destination")

In [0]:
label_encoder = LabelEncoder()
encoded_y_train = label_encoder.fit_transform(y_train)
encoded_y_test = label_encoder.fit_transform(y_test)

In [0]:
result_exp = pd.read_csv("/content/drive/My Drive/progetto_aml/risultati_AUTOML/risultati_SMBO_exp_XGB.csv")
result_prob = pd.read_csv("/content/drive/My Drive/progetto_aml/risultati_AUTOML/risultati_SMBO_prob_XGB.csv")

In [0]:
hyper_exp = result_exp.sort_values(by="ndcg", ascending=False).head(1).to_dict("r")[0]
hyper_prob = result_prob.sort_values(by="ndcg", ascending=False).head(1).to_dict("r")[0]

In [0]:
xgb = XGBClassifier(max_depth=int(hyper_prob["max_depth"]), learning_rate=hyper_prob["lr"], n_estimators=int(hyper_prob["n_estimator"]),
                    objective='multi:softprob', colsample_bytree=1, 
                    seed=42, n_jobs = -1, tree_method = "gpu_hist", max_delta_step = int(hyper_prob["max_delta_step"]))

In [0]:
xgb.fit(train.values, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.28681676944118395, max_delta_step=7, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=39, n_jobs=-1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
              silent=None, subsample=1, tree_method='gpu_hist', verbosity=1)

In [0]:
y_pred_prob = xgb.predict_proba(X_test.values)

In [0]:
ndcg_score(encoded_y_test, y_pred_prob, k = 5)

0.8312148141429794

In [0]:
id_test = test.pop("id")
test.pop("country_destination")

0        -99
1        -99
2        -99
3        -99
4        -99
        ... 
62091    -99
62092    -99
62093    -99
62094    -99
62095    -99
Name: country_destination, Length: 62096, dtype: object

In [0]:
prob_test = xgb.predict_proba(test.values)

# Creazione file da consegnare su Kaggle

In [0]:
ids = []  #list of ids
cts = []  #list of countries
for i in range(len(id_test)):
    idx = id_test[i]
    ids += [idx] * 5
    cts += label_encoder.inverse_transform(np.argsort(prob_test[i])[::-1])[:5].tolist()

In [0]:
sub = pd.DataFrame(np.column_stack((ids, cts)), columns=['id', 'country'])

In [0]:
sub.to_csv("sub_automl.csv", index=False)